# Data exploration notebook for the SIDER datasete

## 1. Import dataset

In [ ]:
from pathlib import Path
import subprocess

# Chemin vers le dossier data (au même niveau que regression)
# Path.cwd() correspond au dossier depuis lequel tu as lancé le notebook
data_dir = Path.cwd().parent / "data"

# Exemple : vérifier si un fichier spécifique est présent
sider_file = data_dir / "sider_dataset.csv"

# Si les fichiers n'existent pas, lancer le script de téléchargement
if not sider_file.exists():
    print("Datasets non trouvés, téléchargement en cours...")
    # On suppose que le script download_dataset.py est à la racine du projet
    subprocess.run(["python", str(Path.cwd().parent / "src" / "download_dataset.py")])
else:
    print("Datasets déjà présents.")


Datasets non trouvés, téléchargement en cours...
